I am starting with downloading the bulk of the packages that will be used to run this project.  some will prob not get used in these innitial steps, but they have all been part of the course thus far and will likely get used over the next few exercises.  I have used the notes from the course as to what each one does.

In [1]:
from bs4 import BeautifulSoup # library for web scraping
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [2]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


Below and I getting the data from the wiki page as instructed.  

In [3]:
wiki = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(wiki.text, 'lxml')

I have the data and am now converting it to data frames - I printed the data frame to make sure it worked

In [4]:
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
df = pd.DataFrame(data = data,columns = columns)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
df = df[df['Borough'] != 'Not assigned'] #this is a simple way to remove the "not assigned buroughs"

In [6]:
df= df.groupby('Postal Code').agg(lambda x: ','.join(x)) # using group by function and agg to concatenate 

In [7]:
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough'] #replaceing not assinged wiht burigon names

In [8]:
df.head(20) #showing the final output of the frame.  I expanded the using head to 20 rows. Looks good to me.

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [9]:
df.shape # here is the answer to the first question.  

(103, 2)

the next sections are the downloaded of teh cooridiantes and the merging of the two data frame - I was lazy and simply downloaded a csv as that is the simplest and fastest way to get the data to merge.

In [10]:
data=pd.read_csv('Geospatial_Coordinates.csv', header=0) #i downloaded the csv and uploaded it as a data frame

In [11]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
comp_df = pd.merge(left=df, right=data, left_on='Postal Code', right_on='Postal Code') #easy merge 

In [13]:
comp_df.shape

(103, 5)

In [18]:
comp_df.head() #data check - looks good

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


we are now on to part 3 - the exploratory analysis. I am going to look at toronto as instructed.

In [20]:
CLIENT_ID = 'KARNGMJWOURD3LZYPRHGS3IEKZUZQF351EMRPWCYLI1BLKRJ' # your Foursquare ID
CLIENT_SECRET = 'JXEXLAJQH3JQ2PFEWFPF1GJ21OILUCMHAEL0Y5MTPQTWCJT1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KARNGMJWOURD3LZYPRHGS3IEKZUZQF351EMRPWCYLI1BLKRJ
CLIENT_SECRET:JXEXLAJQH3JQ2PFEWFPF1GJ21OILUCMHAEL0Y5MTPQTWCJT1


In [21]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


I prefer Thai to itallian, so I'm going to searhc for that... 

In [39]:
##I'm setting the query to look for thai - just like we did in the NY exercize.  

search_query = 'Thai'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Thai .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=KARNGMJWOURD3LZYPRHGS3IEKZUZQF351EMRPWCYLI1BLKRJ&client_secret=JXEXLAJQH3JQ2PFEWFPF1GJ21OILUCMHAEL0Y5MTPQTWCJT1&ll=43.6534817,-79.3839347&v=20180604&query=Thai&radius=500&limit=30'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f22222bcc2d4f2e20496fe0'},
 'response': {'venues': [{'id': '4b8ee6c9f964a520a03d33e3',
    'name': 'Thai Express',
    'location': {'address': '218 Yonge St, CF Toronto Eaton Centre',
     'crossStreet': 'in Toronto Eaton Centre',
     'lat': 43.653224,
     'lng': -79.380327,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.653224,
       'lng': -79.380327}],
     'distance': 291,
     'postalCode': 'M5B 2H6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['218 Yonge St, CF Toronto Eaton Centre (in Toronto Eaton Centre)',
      'Toronto ON M5B 2H6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15960

In [42]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()
#below is a list of all of the thai venues

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4b8ee6c9f964a520a03d33e3,Thai Express,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1596072480,False,"218 Yonge St, CF Toronto Eaton Centre",in Toronto Eaton Centre,43.653224,-79.380327,"[{'label': 'display', 'lat': 43.653224, 'lng':...",291,M5B 2H6,CA,Toronto,ON,Canada,"[218 Yonge St, CF Toronto Eaton Centre (in Tor..."
1,4b043153f964a520f85122e3,Thai Express,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1596072480,False,505 University Ave,NaN,43.655835,-79.388130,"[{'label': 'display', 'lat': 43.655835, 'lng':...",427,M5G 1X3,CA,Toronto,ON,Canada,"[505 University Ave, Toronto ON M5G 1X3, Canada]"
2,4cb4c5a255af224ba3b58d7e,Thai Island,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1596072480,False,333 Bay St.,at Adelaide St. W,43.650567,-79.379534,"[{'label': 'display', 'lat': 43.65056717656215...",480,NaN,CA,Toronto,ON,Canada,"[333 Bay St. (at Adelaide St. W), Toronto ON, ..."
3,4b5a3107f964a5202bb328e3,Thai Express,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1596072480,False,40 King Street West #130,Yonge,43.649739,-79.378925,"[{'label': 'display', 'lat': 43.6497393194015,...",579,M5H 3Y2,CA,Toronto,ON,Canada,"[40 King Street West #130 (Yonge), Toronto ON ..."
4,4b81b646f964a52056b930e3,Thai One On - Authentic Thai & Maylasian Cuisine,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",v-1596072480,False,142 Dundas Street West,NaN,43.655470,-79.384998,"[{'label': 'display', 'lat': 43.65547, 'lng': ...",237,NaN,CA,Toronto,ON,Canada,"[142 Dundas Street West, Toronto ON, Canada]"


In [43]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Thai Express,Restaurant,"218 Yonge St, CF Toronto Eaton Centre",in Toronto Eaton Centre,43.653224,-79.380327,"[{'label': 'display', 'lat': 43.653224, 'lng':...",291,M5B 2H6,CA,Toronto,ON,Canada,"[218 Yonge St, CF Toronto Eaton Centre (in Tor...",4b8ee6c9f964a520a03d33e3
1,Thai Express,Restaurant,505 University Ave,NaN,43.655835,-79.388130,"[{'label': 'display', 'lat': 43.655835, 'lng':...",427,M5G 1X3,CA,Toronto,ON,Canada,"[505 University Ave, Toronto ON M5G 1X3, Canada]",4b043153f964a520f85122e3
2,Thai Island,Asian Restaurant,333 Bay St.,at Adelaide St. W,43.650567,-79.379534,"[{'label': 'display', 'lat': 43.65056717656215...",480,NaN,CA,Toronto,ON,Canada,"[333 Bay St. (at Adelaide St. W), Toronto ON, ...",4cb4c5a255af224ba3b58d7e
3,Thai Express,Thai Restaurant,40 King Street West #130,Yonge,43.649739,-79.378925,"[{'label': 'display', 'lat': 43.6497393194015,...",579,M5H 3Y2,CA,Toronto,ON,Canada,"[40 King Street West #130 (Yonge), Toronto ON ...",4b5a3107f964a5202bb328e3
4,Thai One On - Authentic Thai & Maylasian Cuisine,Thai Restaurant,142 Dundas Street West,NaN,43.655470,-79.384998,"[{'label': 'display', 'lat': 43.65547, 'lng': ...",237,NaN,CA,Toronto,ON,Canada,"[142 Dundas Street West, Toronto ON, Canada]",4b81b646f964a52056b930e3
5,Thai On Yonge,Thai Restaurant,370 Yonge Street,Yonge and Gerrard,43.658607,-79.381999,"[{'label': 'display', 'lat': 43.65860730950852...",591,NaN,CA,Toronto,ON,Canada,"[370 Yonge Street (Yonge and Gerrard), Toronto...",543da069498e0fa6d2895ab4
6,Evergreen Thai,Thai Restaurant,175 Dundas Street West,NaN,43.655174,-79.386326,"[{'label': 'display', 'lat': 43.65517362870162...",269,M5G,CA,Toronto,ON,Canada,"[175 Dundas Street West, Toronto ON M5G, Canada]",4ad8b71cf964a520ed1321e3
7,Ruby Thai,Thai Restaurant,220 Yonge St.,"in Urban Eatery, Toronto Eaton Centre",43.655239,-79.380797,"[{'label': 'display', 'lat': 43.65523934091542...",319,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St. (in Urban Eatery, Toronto Eaton...",4e67e86bfa76f38efb5281c8
8,Thai Bright,Thai Restaurant,109 McCaul St,NaN,43.653802,-79.390739,"[{'label': 'display', 'lat': 43.65380179347436...",549,NaN,CA,Toronto,ON,Canada,"[109 McCaul St, Toronto ON, Canada]",509d9368e4b0eaa915689a6b
9,California Thai,Thai Restaurant,10 Dundas St. E,at Yonge St.,43.656367,-79.380978,"[{'label': 'display', 'lat': 43.65636652255975...",399,M5B 2G9,CA,Toronto,ON,Canada,"[10 Dundas St. E (at Yonge St.), Toronto ON M5...",4bc8fa3a511f95218cd9acc7


In [44]:
dataframe_filtered.name

0                                         Thai Express
1                                         Thai Express
2                                          Thai Island
3                                         Thai Express
4     Thai One On - Authentic Thai & Maylasian Cuisine
5                                        Thai On Yonge
6                                       Evergreen Thai
7                                            Ruby Thai
8                                          Thai Bright
9                                      California Thai
10                                         Thai Island
11                                       New Thai Food
12                                          Salad King
Name: name, dtype: object

In [45]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the city hall

# add a red circle marker to represent the City Hall
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Thai restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [46]:
venue_id = '4b81b646f964a52056b930e3' # ID of Thai One On - Authentic Thai & Maylasian Cuisine - seems like a non-chain
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url



'https://api.foursquare.com/v2/venues/4b81b646f964a52056b930e3?client_id=KARNGMJWOURD3LZYPRHGS3IEKZUZQF351EMRPWCYLI1BLKRJ&client_secret=JXEXLAJQH3JQ2PFEWFPF1GJ21OILUCMHAEL0Y5MTPQTWCJT1&v=20180604'

In [47]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])


{'id': '4b81b646f964a52056b930e3',
 'name': 'Thai One On - Authentic Thai & Maylasian Cuisine',
 'contact': {},
 'location': {'address': '142 Dundas Street West',
  'lat': 43.65547,
  'lng': -79.384998,
  'labeledLatLngs': [{'label': 'display', 'lat': 43.65547, 'lng': -79.384998}],
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['142 Dundas Street West', 'Toronto ON', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/thai-one-on--authentic-thai--maylasian-cuisine/4b81b646f964a52056b930e3',
 'categories': [{'id': '4bf58dd8d48988d149941735',
   'name': 'Thai Restaurant',
   'pluralName': 'Thai Restaurants',
   'shortName': 'Thai',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/thai_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 5},
 'price': {'tier': 2, 'message': 'Moderate', 'currency': '$'},
 'likes': {'count': 0, 'groups': []},
 'dislike': False,
 'ok': False,
 'allow

In [48]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


No rating?? I'll be the first to give them a try!  First I'll check the tips

In [49]:
result['response']['venue']['tips']['count']

5

In [50]:
## Ecco Tips
limit = 5 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2224bbb337623dc2a9d73f'},
 'response': {'tips': {'count': 5,
   'items': [{'id': '4f1734bee4b0f22afc5d0dcf',
     'createdAt': 1326920894,
     'text': 'Amazingly flavourful dishes!! Ordering for a large party?? Try one of their combos and pre-set meals when you order online at Just-Eat.ca for pick-up or delivery!',
     'type': 'user',
     'url': 'http://www.just-eat.ca/restaurants-thai-one-on-dundas-west',
     'canonicalUrl': 'https://foursquare.com/item/4f1734bee4b0f22afc5d0dcf',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '17804224',
      'firstName': 'Just-Eat.ca',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/VUBDKLDS2TH2TP1G.jpg'}}}]}}}

In [51]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'url', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

Now I'll just downtown torronto

In [54]:
latitude = 43.645711
longitude = -79.392732

In [55]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=KARNGMJWOURD3LZYPRHGS3IEKZUZQF351EMRPWCYLI1BLKRJ&client_secret=JXEXLAJQH3JQ2PFEWFPF1GJ21OILUCMHAEL0Y5MTPQTWCJT1&ll=43.645711,-79.392732&v=20180604&radius=500&limit=30'

In [57]:
results = requests.get(url).json()
'There are {} around downtown.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around downtown.'

In [58]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5a219ab81c0b34450e104de7',
  'name': 'General Assembly',
  'location': {'address': '331 Adelaide Street West',
   'lat': 43.647326115934106,
   'lng': -79.39304291907733,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.647326115934106,
     'lng': -79.39304291907733}],
   'distance': 181,
   'postalCode': 'M5V',
   'cc': 'CA',
   'neighborhood': 'Entertainment District',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['331 Adelaide Street West',
    'Toronto ON M5V',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1ca941735',
    'name': 'Pizza Place',
    'pluralName': 'Pizza Places',
    'shortName': 'Pizza',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []

In [60]:
###here is a bunch of stuff around the downtown....

dataframe = pd.json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,General Assembly,Pizza Place,331 Adelaide Street West,43.647326,-79.393043,"[{'label': 'display', 'lat': 43.647326115934106, 'lng': -79.39304291907733}]",181,M5V,CA,Entertainment District,Toronto,ON,Canada,"[331 Adelaide Street West, Toronto ON M5V, Canada]",NaN,5a219ab81c0b34450e104de7
1,The Second City,Comedy Club,51 Mercer St,43.645633,-79.391225,"[{'label': 'display', 'lat': 43.64563251724917, 'lng': -79.39122468317156}]",121,M5V 9G9,CA,NaN,Toronto,ON,Canada,"[51 Mercer St (at Blue Jays Way), Toronto ON M5V 9G9, Canada]",at Blue Jays Way,4ad4c05cf964a520c9f520e3
2,SOMA chocolatemaker,Dessert Shop,443 King St. W,43.645328,-79.395714,"[{'label': 'display', 'lat': 43.64532829235515, 'lng': -79.39571438648944}]",243,M5V 1K4,CA,NaN,Toronto,ON,Canada,"[443 King St. W, Toronto ON M5V 1K4, Canada]",NaN,4df796f1aeb7da11e1a974d4
3,Strange Love,Coffee Shop,101 Spadina Ave,43.646301,-79.395184,"[{'label': 'display', 'lat': 43.646301465741814, 'lng': -79.39518379105066}]",208,M5V 2K2,CA,NaN,Toronto,ON,Canada,"[101 Spadina Ave (btwn Adelaide St W & Oxley St), Toronto ON M5V 2K2, Canada]",btwn Adelaide St W & Oxley St,58a1d1a8326c5a3a1c150a60
4,Bar Hop,Beer Bar,391 King St W,43.645603,-79.393583,"[{'label': 'display', 'lat': 43.64560340159375, 'lng': -79.3935826420784}]",69,M5V 1K1,CA,NaN,Toronto,ON,Canada,"[391 King St W (at Charlotte St), Toronto ON M5V 1K1, Canada]",at Charlotte St,4fcc0dd177167574f0de38d5
5,TIFF Bell Lightbox,Movie Theater,350 King St. W,43.646342,-79.390063,"[{'label': 'display', 'lat': 43.64634159071232, 'lng': -79.39006297129853}]",226,M5V 3X5,CA,Entertainment District,Toronto,ON,Canada,"[350 King St. W (at John St.), Toronto ON M5V 3X5, Canada]",at John St.,4bcf714ab221c9b67f0ad2d0
6,Le Germain Hotel Toronto Mercer,Hotel,30 Mercer St,43.645756,-79.390904,"[{'label': 'display', 'lat': 43.64575555057125, 'lng': -79.39090442494489}]",147,M5V 1H3,CA,NaN,Toronto,ON,Canada,"[30 Mercer St, Toronto ON M5V 1H3, Canada]",NaN,4ad4c05cf964a520baf520e3
7,Akira Back,Japanese Restaurant,80 Blue Jays Way,43.645376,-79.392063,"[{'label': 'display', 'lat': 43.645375789305994, 'lng': -79.39206284397356}]",65,M5V 2G3,CA,NaN,Toronto,ON,Canada,"[80 Blue Jays Way, Toronto ON M5V 2G3, Canada]",NaN,5a2c772450a6f0229d83d923
8,Totum Life Science King,Gym,101-445 King St W,43.645119,-79.395883,"[{'label': 'display', 'lat': 43.645119, 'lng': -79.395883}]",262,M5V 1K4,CA,NaN,Toronto,ON,Canada,"[101-445 King St W (at Spadina Ave.), Toronto ON M5V 1K4, Canada]",at Spadina Ave.,4ae49933f964a520289c21e3
9,Khao San Road,Thai Restaurant,11 Charlotte St.,43.646301,-79.393811,"[{'label': 'display', 'lat': 43.64630148944898, 'lng': -79.39381131495668}]",108,NaN,CA,Entertainment District,Toronto,ON,Canada,"[11 Charlotte St. (King St. W), Toronto ON, Canada]",King St. W,589fafe634935555b0fa6c35


In [61]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

Now i'm looking for the trending venues... becuas I'm a trendy guy.  what data scientist aren't trendy?  am i right???

In [62]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f22268285d1da39f9a363b0'},
 'response': {'venues': []}}

In [63]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [64]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

In [ ]:
###well that no good?  how are there not trending venues in downtown toronto?  I guess its cold...
### at this point I would have mappped teh trending venues, but as you can see toronto isn't a trending place... sad

In [65]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
        
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'